In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import time
from binance.client import Client
import ccxt

# Binance Futures API keys
API_Key = 'e80b538d742b5c39934cb6303bcf71b3fcd148e5d948939fee75a517934edc06'
Secret_Key = '2d1b453e5bee3cfadc332541533993232679fd58bed50205036184ed10644efc'

binance = ccxt.binance({'apiKey':API_Key, 'secret':Secret_Key, 'options':{'defaultType':'future'}})
binance.set_sandbox_mode(True)

In [2]:
binance.fetch_balance()

{'info': {'totalInitialMargin': '6153.49800000',
  'totalMaintMargin': '49.22798400',
  'totalWalletBalance': '12259.63677570',
  'totalUnrealizedProfit': '110.40039999',
  'totalMarginBalance': '12370.03717569',
  'totalPositionInitialMargin': '6153.49800000',
  'totalOpenOrderInitialMargin': '0.00000000',
  'totalCrossWalletBalance': '12259.63677570',
  'totalCrossUnPnl': '110.40039999',
  'availableBalance': '6210.85817569',
  'maxWithdrawAmount': '6210.85817569',
  'assets': [{'asset': 'FDUSD',
    'walletBalance': '0.00000000',
    'unrealizedProfit': '0.00000000',
    'marginBalance': '0.00000000',
    'maintMargin': '0.00000000',
    'initialMargin': '0.00000000',
    'positionInitialMargin': '0.00000000',
    'openOrderInitialMargin': '0.00000000',
    'crossWalletBalance': '0.00000000',
    'crossUnPnl': '0.00000000',
    'availableBalance': '0.00000000',
    'maxWithdrawAmount': '0.00000000',
    'updateTime': '0'},
   {'asset': 'BNB',
    'walletBalance': '0.00000000',
    '

In [3]:
# Symbol and trading parameters
symbol = 'BTC/USDT'
timeframe = '1d'
base_lot_size = 0.5  # Base position size without leverage
leverage = 2 #CHANGE SIZE TO BE CONSISTENTLY 15000, MAYBE 7500 X2

In [4]:
# Load markets and set leverage
try:
    markets = binance.load_markets()
    binance.set_leverage(leverage, symbol)
    print(f"Leverage set to {leverage}x for {symbol}")
except Exception as e:
    print(f"Error setting leverage: {e}")

def fetch_ohlcv():
    """Fetch historical OHLCV data"""
    try:
        bars = binance.fetch_ohlcv(symbol, timeframe, limit=50)
        df = pd.DataFrame(bars, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        return df
    except Exception as e:
        print(f"Error fetching OHLCV data: {e}")
        return pd.DataFrame()

def ac_indicator(df, short_sma_period=5, long_sma_period=34, ao_sma_period=5):
    """Calculate Accelerator Oscillator (AC)"""
    df['Median_Price'] = (df['High'] + df['Low']) / 2
    df['SMA_Short'] = df['Median_Price'].rolling(window=short_sma_period).mean()
    df['SMA_Long'] = df['Median_Price'].rolling(window=long_sma_period).mean()
    df['AO'] = df['SMA_Short'] - df['SMA_Long']
    df['SMA_AO'] = df['AO'].rolling(window=ao_sma_period).mean()
    df['AC'] = df['AO'] - df['SMA_AO']
    df.dropna(inplace=True)
    return df

Leverage set to 2x for BTC/USDT


In [5]:
def check_positions():
    """Check if there's an open position for the symbol using multiple methods"""
    try:
        # Method 1: Check using fetch_positions
        positions = binance.fetch_positions()
        symbol_normalized = symbol.replace('/', '')
        
        for pos in positions:
            pos_symbol = pos['symbol']
            # Check various symbol formats
            possible_formats = [symbol_normalized, symbol, symbol + ":USDT", symbol.replace('/', '') + "USDT"]
            if any(fmt in pos_symbol for fmt in possible_formats):
                # Determine position amount - ensure it's properly extracted
                position_amt = float(pos.get('positionAmt', 0)) or float(pos.get('contracts', 0))
                
                # Determine position side - check for explicit side field first
                side = pos.get('side') or ('long' if position_amt > 0 else 'short')
                
                if abs(position_amt) > 0.00001:  # Check if position size is significant
                    print(f"Found active position: {pos_symbol}, Amount: {position_amt}, Side: {side}")
                    # Add the detected side to the position data for later use
                    pos['detected_side'] = side
                    return pos
        
        # Method 2: Check using fetch_balance for futures positions
        balance = binance.fetch_balance()
        if 'info' in balance and 'positions' in balance['info']:
            for pos in balance['info']['positions']:
                if symbol.replace('/', '') in pos['symbol']:
                    position_amt = float(pos.get('positionAmt', 0))
                    side = pos.get('positionSide', 'BOTH')
                    # Determine side based on either explicit side or amount
                    detected_side = side if side != 'BOTH' else ('long' if position_amt > 0 else 'short')
                    
                    if abs(position_amt) > 0.00001:
                        print(f"Found active position via balance: {pos['symbol']}, Amount: {position_amt}, Side: {detected_side}")
                        pos['detected_side'] = detected_side
                        return pos
        
        print("No active positions found after thorough checking")
        return None
    except Exception as e:
        print(f"Error checking positions: {e}")
        return None
def place_order(order_type, stop_loss_pct=0.02):
    """Place a market order with dynamic lot size based on account balance,
    ensuring minimum notional value."""
    try:
        balance = binance.fetch_balance()
        total_balance = balance['total']['USDT'] if 'USDT' in balance['total'] else 0
        risk_percentage = 1  # Risking 1% per trade
        risk_amount = total_balance * risk_percentage

        ticker = binance.fetch_ticker(symbol)
        market_price = ticker['last']

        # Calculate the initial quantity
        initial_quantity = risk_amount / market_price

        # Calculate the notional value of the initial order
        notional_value = initial_quantity * market_price

        # Adjust the quantity if the notional value is too small
        if notional_value < 100:
            # Calculate the minimum quantity needed to reach 100 USDT notional value
            min_quantity = 100 / market_price
            quantity_to_trade = min_quantity
            print(f"Warning: Calculated notional value ({notional_value:.2f} USDT) is below minimum. Adjusting trade size to {quantity_to_trade:.8f} to meet the 100 USDT minimum.")
        else:
            quantity_to_trade = initial_quantity

        # Ensure we are trading a positive amount
        if quantity_to_trade <= 0:
            print("Error: Calculated trade quantity is zero or negative. Skipping order.")
            return

        # Create main order
        order = binance.create_order(
            symbol=symbol,
            type='market',
            side=order_type,
            amount=quantity_to_trade
        )
        print(f"{order_type.capitalize()} order placed: {order}")

        # Place stop-loss order
        sl_side = 'sell' if order_type == 'buy' else 'buy'
        sl_order = binance.create_order(
            symbol=symbol,
            type='stop_market',
            side=sl_side,
            amount=quantity_to_trade,
            params={'stopPrice': market_price * (1 - stop_loss_pct) if order_type == 'buy' else market_price * (1 + stop_loss_pct)}
        )
        print(f"Stop-loss placed: {sl_order}")

        # Send email notification
        send_email(
            f"Trade Alert: {order_type.upper()} Position Opened",
            f"Entered {order_type} position for {symbol} with size {quantity_to_trade:.8f} at {market_price}.\n"
            f"Stop-loss set at {market_price * (1 - stop_loss_pct) if order_type == 'buy' else market_price * (1 + stop_loss_pct)}."
        )
    except Exception as e:
        print(f"Error placing order: {e}")
        send_email("Order Error", f"Failed to place {order_type} order: {e}")
    # Add this at the end of the function:
    print("Waiting for position to be registered...")
    time.sleep(2)  # Wait 2 seconds for the exchange to process the order


def close_position():
    """Close an open position"""
    try:
        position = check_positions()
        if position:
            # Get position details
            position_amt = abs(float(position.get('positionAmt', 0)) or float(position.get('contracts', 0)) or 0)
            
            # Get the explicitly detected side
            position_side = position.get('detected_side', '').lower()
            
            # Determine close side based on position side
            if position_side == 'long':
                close_side = 'sell'
            elif position_side == 'short':
                close_side = 'buy'
            else:
                # Fallback if side detection fails
                raw_amt = float(position.get('positionAmt', 0)) or float(position.get('contracts', 0))
                close_side = 'sell' if raw_amt > 0 else 'buy'
            
            # Ensure position amount is valid
            if position_amt < 0.001:  # Minimum amount for BTC
                position_amt = 0.001  # Set to minimum tradable amount
            
            print(f"Closing {position_side} position with {close_side} order for {position_amt} units")
            
            # Create market order to close position
            order = binance.create_order(
                symbol=symbol,
                type='market',
                side=close_side,
                amount=position_amt
            )
            print(f"Position closed: {order}")
            send_email("Position Closed", f"Closed {symbol} {position_side} position of {position_amt} units")
            return True
        else:
            print("No position to close")
            return False
    except Exception as e:
        print(f"Error closing position: {e}")
        send_email("Close Position Error", f"Failed to close position: {e}")
        return False

In [6]:
def debug_position_format():
    """Debug function to examine the exact format of position data"""
    try:
        # Method 1
        positions1 = binance.fetch_positions()
        print(f"===== fetch_positions() =====")
        for pos in positions1:
            if symbol.replace('/', '') in str(pos):
                print(f"Found with method 1: {pos}")
                print(f"Keys: {pos.keys()}")
        
        # Method 2
        balance = binance.fetch_balance()
        print(f"===== fetch_balance() positions =====")
        if 'info' in balance and 'positions' in balance['info']:
            for pos in balance['info']['positions']:
                if symbol.replace('/', '') in str(pos):
                    print(f"Found with method 2: {pos}")
                    print(f"Keys: {pos.keys() if hasattr(pos, 'keys') else 'Not a dict'}")
        
        # Method 3
        try:
            positions3 = binance.fapiPrivateGetPositionRisk()
            print(f"===== fapiPrivateGetPositionRisk() =====")
            for pos in positions3:
                if symbol.replace('/', '') in str(pos):
                    print(f"Found with method 3: {pos}")
                    print(f"Keys: {pos.keys() if hasattr(pos, 'keys') else 'Not a dict'}")
        except Exception as e:
            print(f"Method 3 error: {e}")
    
    except Exception as e:
        print(f"Debug error: {e}")


In [7]:
import smtplib
import time
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(subject, body):
    sender_email = "henryparkin21@gmail.com"
    receiver_email = "henryparkin21@gmail.com"
    password = "wyqv fnpi nttm geiy"
    
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)  # Update SMTP server & port
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
    except Exception as e:
        print(f"Email failed to send: {e}")


def trading_bot(single_cycle=False):
    """Main trading loop
    
    Args:
        single_cycle (bool): If True, run only one cycle and then exit
    """
    print("Starting trading bot...")
    send_email("Script Started", "The trading bot script has been called and is now running.")

    while True:
        try:
            # Check account balance for monitoring
            balance = binance.fetch_balance()
            total_balance = balance['total']['USDT'] if 'USDT' in balance['total'] else 0
            print(f"Current account balance: {total_balance} USDT")
            
            # Fetch and analyze market data
            df = fetch_ohlcv()
            if df.empty:
                print("No data received, skipping cycle")
                if single_cycle:
                    print("Single cycle mode: Bot shutting down after one cycle.")
                    break
                time.sleep(60)
                continue
                
            df = ac_indicator(df)
            
            if len(df) < 2:
                print("Not enough data points for analysis")
                if single_cycle:
                    print("Single cycle mode: Bot shutting down after one cycle.")
                    break
                time.sleep(60)
                continue
                
            latest_AO = df['AO'].iloc[-1]
            latest_AC = df['AC'].iloc[-1]
            
            print(f"Latest indicators - AO: {latest_AO}, AC: {latest_AC}")
            
            # Check current positions
            position = check_positions()
            
            if position:
                # Get the explicitly detected side from check_positions
                position_side = position.get('detected_side', 'unknown').upper()
                position_amt = abs(float(position.get('positionAmt', 0))) or abs(float(position.get('contracts', 0)))
                
                # Only use the position_amt sign for determining side if we don't have explicit side info
                if position_side == 'UNKNOWN':
                    raw_amt = float(position.get('positionAmt', 0)) or float(position.get('contracts', 0))
                    position_side = "LONG" if raw_amt > 0 else "SHORT"
                
                print(f"Current position: {position_side} {position_amt} units")
                
                # Check if position should be closed based on indicators
                if (latest_AO < latest_AC and position_side == "LONG") or (latest_AO > latest_AC and position_side == "SHORT"):
                    print("Signal to close position detected")
                    close_position()
            else:
                print("No current position, checking for entry signals")
                # Only open new positions if none exist
                if latest_AO > latest_AC:
                    print("Bullish signal detected - entering LONG")
                    place_order('buy')
                elif latest_AO < latest_AC:
                    print("Bearish signal detected - entering SHORT")
                    place_order('sell')
                else:
                    print("No clear signal, staying out of market")
            
            # If single cycle mode is enabled, exit after one complete cycle
            if single_cycle:
                print("Single cycle mode: Bot shutting down after one cycle.")
                break
                
        except Exception as e:
            print(f"Error in trading loop: {e}")
            send_email("Bot Error", f"Trading bot encountered an error: {e}")
            
            # Even in case of error, exit if single cycle mode is enabled
            if single_cycle:
                print("Single cycle mode: Bot shutting down after error.")
                break
        
        print(f"Sleeping for 60 seconds before next cycle...")
        time.sleep(60)
    
    print("Bot has completed execution.")
    send_email("Bot Stopped", "The trading bot has completed its execution.")



In [8]:
# Entry point if running script directly
if __name__ == "__main__":
    try:
        # Run the bot with single_cycle=True for testing or False for continuous operation
        trading_bot(single_cycle=True)  # Change to False for continuous operation
    except Exception as e:
        print(f"Fatal error in main execution: {e}")
        send_email("Fatal Bot Error", f"Trading bot encountered a fatal error: {e}")

Starting trading bot...
Current account balance: 12370.05139091 USDT
Latest indicators - AO: 7695.215882352961, AC: -277.80258823527583
Found active position: BTC/USDT:USDT, Amount: 0.13, Side: long
Current position: LONG 0.13 units
Single cycle mode: Bot shutting down after one cycle.
Bot has completed execution.
